# Генерация ДНК последовательности с помощью марковской цепи

В качестве референсного генома возьмем геном 13 хромосомы человека (но только часть).

Файл скачан отсюда: [link](http://hgdownload.cse.ucsc.edu/goldenPath/hg38/chromosomes/)

## Нахождение матрицы переходов

Подсчет частот встречаемости пар нуклеотидов для создания матрицы переходов для создания Цепи Маркова.

In [7]:
!head generating_dna/chr13_KI270838v1_alt.fa

>chr13_KI270838v1_alt
GTAGGGACGACCTAGATGGAATCGGAGGGCATTGGGAGCATTCAGCACAG
AGGCTTTCAAACGTGGGGCACTTTCTTTGGATGGAAGCAAAGGGGAAACC
CCCCGAAGTCCCCCCGCACGGCTGCCATGCAGGCTCAGGGTCAGGTTCTC
CTCCTTCAAGGCTGGAGTGCCTGGGCTCCATCGTCAGCCTCCTTCTCCAC
CTGCGTTGGTTCCTTAGCCAAAGCCATCCAGTCCGGGCCTGCAGCTGGGA
GACAGGGAAGAAGAGGATGTGTGGGGCCCGTGGCACTTGACCTGGGTGAC
ATGAGGGTCACTTCAAACACGGCCAGTGTGCAGCCTCCCAGGTCCCTGAG
GTCGGCGTCTCCGGGTCAACCCTTAGTGACATCTCCAGGCCCAGGTGGCC
TTGAAGTGTGGACACAGGTGGGCACCACTGGGACAAGCAGCCGTTCCCAC


In [17]:
with open('generating_dna/chr13_KI270838v1_alt.fa') as fn:
    lines = fn.readlines()[1:]
data = ''.join([line.strip().upper() for line in lines])
print('Количество нуклеотидов в файле: ', len(data))

Количество нуклеотидов в файле:  306913


Для подсчета биграм удобно использовать nltk пакет.

Он разбивает всю строку на перекрывающиеся биграмы.

Затем подсчитываем количество таких биграм с помощью Counter.

In [27]:
import nltk
from nltk import bigrams

string_bigrams = bigrams(data)

In [18]:
from collections import Counter

In [28]:
c = Counter(string_bigrams)

In [34]:
c

Counter({('A', 'A'): 24653,
         ('A', 'C'): 16332,
         ('A', 'G'): 23049,
         ('A', 'T'): 19623,
         ('C', 'A'): 25437,
         ('C', 'C'): 20474,
         ('C', 'G'): 4863,
         ('C', 'T'): 20715,
         ('G', 'A'): 18902,
         ('G', 'C'): 16086,
         ('G', 'G'): 20221,
         ('G', 'T'): 15668,
         ('T', 'A'): 14665,
         ('T', 'C'): 18598,
         ('T', 'G'): 22743,
         ('T', 'T'): 24883})

In [40]:
# Переведем частоты в вероятности 

from pprint import pprint
transition_dict = {
    'A':{},
    'T':{},
    'G':{},
    'C':{}
}

for pair, freq in c.items():
    transition_dict[pair[0]][pair[1]] = freq

for k, voc in transition_dict.items():
    norm_sum = sum(voc.values())
    for k1, freq in voc.items():
        transition_dict[k][k1] = freq/norm_sum

pprint(transition_dict)

{'A': {'A': 0.2946914185304278,
       'C': 0.1952257432133593,
       'G': 0.2755178885209845,
       'T': 0.23456494973522837},
 'C': {'A': 0.35581697883590485,
       'C': 0.2863937109205612,
       'G': 0.0680244513141882,
       'T': 0.2897648589293458},
 'G': {'A': 0.2666873597923163,
       'C': 0.22695655854508515,
       'G': 0.2852970639276493,
       'T': 0.22105901773494926},
 'T': {'A': 0.18129782788759904,
       'C': 0.22992001384613484,
       'G': 0.281163075325446,
       'T': 0.30761908294082013}}


In [44]:
mapping = {0:'A', 1:'T', 2:'G', 3:'C'}

# Непосредственная генерация

In [42]:
# распределение изначальных
# вероятностей нахождения в состояниях A,T,G,C
# возьмем равновероятными
initial_probabilities = [0.25]*4
initial_probabilities

[0.25, 0.25, 0.25, 0.25]

Каждый столбец `j` матрицы переходов задает распределение вероятностей перехода из состояния `j` (согласно `mapping`) во все 4 состояния.

Т.е. сумма по столбцу в `transition_matrix` равна 1. 

In [48]:
import numpy as np
transition_matrix = np.zeros((4,4))

for j in range(4):
    for i in range(4):
        transition_matrix[i][j] = transition_dict[mapping[j]][mapping[i]]
transition_matrix

array([[ 0.29469142,  0.18129783,  0.26668736,  0.35581698],
       [ 0.23456495,  0.30761908,  0.22105902,  0.28976486],
       [ 0.27551789,  0.28116308,  0.28529706,  0.06802445],
       [ 0.19522574,  0.22992001,  0.22695656,  0.28639371]])

Используем `np.random.choice()` для сэмплирования из распределения.

Исходя из Марковского предположения, на каждой итерации текущий вектор распределений $P_n$ определяется лишь предыдущим состоянием:

In [59]:
sequence_length = 30
P = initial_probabilities

for i in range(sequence_length):
    letter = np.random.choice(np.arange(4), p=P)
    print(mapping[letter],end='', sep='')
    P = transition_matrix[:, letter]

CTCCGGACAGTGGACCAACCTGGTGCTGGG